# Embeddings con LSTM



In [29]:
import torch
import torch.nn as nn
from collections import Counter
import random
from itertools import chain
from pprint import pprint
import numpy as np

In [2]:
cd ..

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal


In [3]:
from helpers.boletines import get_sentences

In [4]:
cd Embeddings/

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


## Definición de constantes

In [5]:
# Batch size durante entrenamiento
batch_size = 32

# Épocas de entrenamiento
num_epochs = 10

# Learning rate for optimizers
dlr = 0.001

# Beta1 hyperparam for Adam optimizer
beta1 = 0.5

# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Semilla a usar en los generadores de números aleatorios
SEED = 42069
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
print("Random Seed: ", SEED)
random.seed(SEED)
torch.manual_seed(SEED)

Random Seed:  42069


Selecciono el tipo de dispositivo a utilizar (gpu o cpu)

In [6]:
# Tipo de datos para los pesos
# dtype = torch.float

# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Obtener corpus

In [7]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [8]:
sentences = get_sentences(uri)

RAC
1. De_la_mano_de jóvenes innovadores , comienza Ricardo_Anaya el camino hacia la Presidencia_de_la_República .
377. Finalmente , sostuvo que López_Obrador es “ el espanta inversiones ” , porque su actitud provoca que estas se vayan , lo cual es muy delicado , porque cuando no hay inversión no crece la economía y no se generan empleos .
753. “ Sí , necesitamos cambiar ciertas piezas de el modelo económico , estamos en un momento crítico , donde tenemos que elegir entre volver a el pasado a_través_de un cómic o construir un futuro más incluyente , igualitario , a_partir_de propuestas series , razonadas , en donde los cómo están claros y los para qué , son parte principal de lo que se pretende hacer ” , enfatizó a el contrastar con la propuesta que hace López_Obrador .
1129. Yo quiero paz para Jalisco , yo quiero paz para México , y esa paz la voy a construir con las mujeres , esa paz la vamos a construir todos juntos ” , enfatizó .

AMLO
1. Asegura AMLO que en campaña todo será amor 

Desenvuelvo los grupos de sentencias en un único arreglo con todas las sentencias de todos los candidatos.

In [9]:
corpus = list(chain(*sentences.values()))

Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [10]:
corpus = [[w.lower() for w in sent] for sent in corpus]
print(f'El corpus consta de {len(corpus)} sentencias.')

El corpus consta de 6889 sentencias.


Selecciono k oraciones de prueba para el modelo

In [11]:
k = 1000
test_sents = random.sample(corpus, k=k)
print('\n'.join([' '.join(sent) for sent in test_sents[::k//10]]))

promete amlo que no cancelará , ni modificará las concesiones en radio y televisión .
además , las estancias que atiendan a ciertos sectores económicos como las maquiladoras , tendrán horario nocturno .
queremos externarle a andrés_manuel que no pierda su buen sentido de el humor , porque , andrés_manuel , ¡ vas a volver a perder ! ” , subrayó meade ante habitantes de la región de los_altos_de_jalisco , quienes se unieron en una sola voz para apoyar a su candidato con la porra ¡ pepe presidente !
meade congrega a más de 60_mil simpatizantes durante los eventos de el fin de semana .
y creo que la conclusión es que no ha fallado la gente , han fallado los gobiernos , porque hemos tenido gobiernos profundamente corruptos y gobiernos profundamente ineficaces y eso es lo que yo he querido cambiar ” .
josé_antonio_meade impulsará el desarrollo integral de niñas , niños y adolescentes , dijo .
este programa , detalló , consiste en acciones concretas para la primera infancia , desde la gestaci

Cuento los tokens en todas las oraciones e imprimo los 40 más frecuentes.

In [12]:
freq = sum([Counter(sent) for sent in test_sents], Counter())
print(freq.most_common()[:-40:-1])

[('toma', 1), ('votarán', 1), ('escribió', 1), ('conciencia', 1), ('plurinominal', 1), ('canadiense', 1), ('adquirido', 1), ('55_millones_de_dólares', 1), ('señalado', 1), ('napoleón_gomez_urrutia', 1), ('contento', 1), ('personalizada', 1), ('formalización', 1), ('misma', 1), ('mueve', 1), ('siglos', 1), ('pasan', 1), ('repite', 1), ('usual', 1), ('interesantes', 1), ('productos', 1), ('comuneros', 1), ('bloqueó', 1), ('vano', 1), ('papantla', 1), ('amecameca', 1), ('slim', 1), ('saliera', 1), ('carlos_slim', 1), ('carlos_salinas', 1), ('posiblemente', 1), ('geografía', 1), ('mapa', 1), ('visita', 1), ('diputado', 1), ('comité_directivo_estatal_de_el_pri', 1), ('presiente', 1), ('mexe', 1), ('reabrir', 1)]


In [13]:
num_tokens = len(freq.keys())
num_tokens

5269

Agrego un diccionario para pasar de palabra a índice numérico

In [14]:
w_to_index = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [15]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = num_tokens + 1
ixEOS = num_tokens + 2
ixUNK = num_tokens + 3

In [16]:
w_to_index[BOS] = ixBOS
w_to_index[EOS] = ixEOS
w_to_index[UNK] = ixUNK

Creo el diccionario inverso, para convertir de índices a palabras

In [17]:
index_to_w = [ w for w, ix in w_to_index.items() ]

In [18]:
# tests = random.sample(w_to_index.keys(), k=10)
# for w in tests:
#     ix = w_to_index[w]
#     assert index_to_w[ix] == w

Defino las variables para la red neuronal

In [19]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = len(index_to_w)

# Dimensión de la capa de embedding
D_emb = 32

# Dimensión de la capa lstm
D_lstm = 16

# Dimensión de la capa de salida
D_out = D_in

## Modelo

In [33]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Embedding') != -1:
        # Regularizo los pesos
        n = m.num_embeddings
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
    elif classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.fill_(0)
    elif classname.find('LSTMCell') != -1:
        n = m.input_size
        y = 1.0/np.sqrt(n)
        m.weight_ih.data.uniform_(-y, y)
        m.weight_hh.data.uniform_(-y, y)
        m.bias_ih.data.fill_(0)
        m.bias_hh.data.fill_(0)

In [34]:
class Model(nn.Module):
    def __init__(self, ngpu):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb),
            nn.LSTMCell(input_size=D_emb, hidden_size=D_lstm, bias=True),
            nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
            nn.Softmax(dim=D_out)
        )

    def forward(self):
        return self.main(input)

In [35]:
model = Model(ngpu).to(device)
model.apply(weights_init)

model

Model(
  (main): Sequential(
    (0): Embedding(2292, 32)
    (1): LSTMCell(32, 16)
    (2): Linear(in_features=16, out_features=2292, bias=True)
    (3): Softmax(dim=2292)
  )
)